In [1]:
import os
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Calibrate camera

### Create pairs of image and object points

In [2]:
out_dir = 'output_images/object_points/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
row_corners = 9
col_corners = 6
objp = np.zeros((row_corners*col_corners,3), np.float32)
objp[:,:2] = np.mgrid[0:row_corners, 0:col_corners].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/cal*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (row_corners,col_corners), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (row_corners,col_corners), corners, ret)
        write_name = out_dir + fname.split('\\')[-1]
        cv2.imwrite(write_name, img)

### Compute calibration matrix

In [3]:
# Do camera calibration given object points and image points
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

### Apply a distortion correction to calibration images

In [4]:
out_dir = 'output_images/calibrated_images/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Make a list of calibration images
images = glob.glob('camera_cal/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite(out_dir + fname.split('\\')[-1],dst)

## Apply a distortion correction to test images

In [5]:
out_dir = 'output_images/undistorted_images/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Make a list of calibration images
images = glob.glob('test_images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite(out_dir + fname.split('\\')[-1],dst)

## Use color transforms, gradients, etc., to create a thresholded binary image

In [6]:
def create_binary_image(img, s_thresh=(90, 255), sx_thresh=(20, 100), l_thresh=(50, 255)):
    """
    Create binary image from a color image by thresholding S and H color channels, as well as the gradient along X axis
    
    Input:
        img - color image
        s_thresh - tuple of minimal and maximal allowable values in S color channel
        l_thresh - tuple of minimal and maximal allowable values in L color channel
        sx_theshold - tuple of minimal and maximal allowable values of gradient along X axis
        
    Output: binary image
    """
    
    img = np.copy(img)
    
    # Convert to HLS color space and separate the H channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    # compute gradient along X axis using Sobel operator
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) 
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold X gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
        
    # Threshold S color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Threshold H color channel
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    # Combine binary images
    combined = np.zeros_like(s_binary)
    combined[(sxbinary == 1) | ((s_binary == 1) & (l_binary == 1))] = 1
    
    return np.uint8(combined)

In [7]:
out_dir = 'output_images/binary_images/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
# Make a list of undistorted images
images = glob.glob('output_images/undistorted_images/*.jpg')

# Create binary image from each undistorted image
for fname in images:
    img = cv2.imread(fname)
    thresholded = create_binary_image(img, s_thresh=(100, 255))
    mpimg.imsave(out_dir+fname.split('\\')[-1], thresholded, cmap='gray')

## Apply a perspective transform to rectify binary image ("birds-eye view")

### Create perspective transform matrix

In [8]:
# define 4 source points 
src = np.float32([[609,439],[669,439],
                  [1046,673],[257,673]])
        
# define 4 destination points 
x1 = 300 
y1 = 0
x2 = 980 
y2 = 720
dst = np.float32([[x1,y1],[x2,y1],[x2,y2],[x1,y2]])
        
# compute perspective transform matrix M
M = cv2.getPerspectiveTransform(src, dst)

# compute inverse perspective transform matrix Minv
Minv = cv2.getPerspectiveTransform(dst, src)

### Apply perspective transform to the test images

In [9]:
# create output directories

# directory of images after perspective transform
if not os.path.exists('output_images/perspective_transform'):
    os.makedirs('output_images/perspective_transform')
    
# directory of original images with the bounding lines of pespective transform     
if not os.path.exists('output_images/binary_images_lines'):
    os.makedirs('output_images/binary_images_lines')
    
# directory of transformed images with the bounding lines of pespective transform 
if not os.path.exists('output_images/perspective_transform_lines'):
    os.makedirs('output_images/perspective_transform_lines')
    
# Make a list of binary images
images = glob.glob('output_images/binary_images/*.jpg')

for fname in images:
    # apply perspective transform
    img_color = cv2.imread(fname)
    img = cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY)
    warped = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    mpimg.imsave('output_images/perspective_transform/'+fname.split('\\')[-1], warped, cmap='gray')
    
    # create an image with the bounding lines of perspective transform
    cv2.polylines(img_color, np.int32([src]), isClosed=True, color=(255,0,0), thickness=2)
    mpimg.imsave('output_images/binary_images_lines/'+fname.split('\\')[-1], img_color)

    # apply perspective transform to the the image with the bounding lines of perspective transform
    warped_lines = cv2.warpPerspective(img_color, M, img_color.shape[1::-1], flags=cv2.INTER_LINEAR)
    mpimg.imsave('output_images/perspective_transform_lines/'+fname.split('\\')[-1], warped_lines)

## Detect lane pixels and fit to find the lane boundary

### Parameters of the pipeline

In [10]:
# Choose the number of sliding windows
nwindows = 20 

# Set the width of the windows +/- margin
margin = 130  

# Set minimum number of pixels found to recenter window
minpix = 10 

# Define conversions in x and y from pixels space to meters
ym_per_pix = 51/720 # meters per pixel in y dimension
xm_per_pix = 3.7/650 # meters per pixel in x dimension

# Smoothing parameter
alpha = 0.5

### Class that represents a detected line

In [11]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, current_fit, x, y):
        
        #polynomial coefficients for the most recent fit
        self.current_fit = current_fit 
        
        # x coordinates of the points that are assigned to the line
        self.x = x 
        
        # y coordinates of the points that are assigned to the line
        self.y = y
        
    def compute_x(self, y_values):
        # compute x value of the fitted line given its y value
        return self.current_fit[0]*y_values**2 + self.current_fit[1]*y_values + self.current_fit[2]

### Auxiliary functions of the pipeline for finding lines

In [12]:
def find_lane_base(img):
    """
    Find line base
    Input: grayscale image
    Output: x coordinates of the base of left and right lines
    """
    
    # Take a histogram of the bottom part of the image
    histogram = np.sum(img[img.shape[0]-200:,:], axis=0)
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    return leftx_base, rightx_base

def track_lane(img, x_base, nonzerox, nonzeroy, out_img, visualize):
    """
    Find entire line using sliding windows
    Input:
        img - original grayscale image
        x_base - x coordinate of the base of the line
        nonzerox - array of x coordinates of non-zero points in the image
        nonzeroy - array of y coordinates of non-zero points in the image
        out_img - color image that is used for visualization
        visualize - indicator if the function needs to visualize the detected line
    Output:
        - Line object of the detected line
        - image that visualizes the line 
    """
        
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    
    # Current positions to be updated for each window
    x_current = x_base
    
    # Create empty lists to receive left and right lane pixel indices
    lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows-5):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        if visualize == True:
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),
                          (0,255,0), 2) 

        # Identify the nonzero pixels in x and y within the window
        good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                     (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]
        
        # Append these indices to the lists
        lane_inds.append(good_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_inds) > minpix:
            x_current = np.int(np.mean(nonzerox[good_inds]))
        
    # Concatenate the arrays of indices
    lane_inds = np.concatenate(lane_inds)
    
    # Extract left and right line pixel positions
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds]  

    # Fit a second order polynomial 
    fit = np.polyfit(y, x, 2)
    
    return Line(fit, x, y), out_img

def radius_of_curvature(x,y,y_eval):
    """
    Compute fit a curve and compute its radius of curvature at a given point
    Input: 
        x - x coordinates of the points of the curve
        y - y coordinates of the points of the curve
        y_eval - y cvoordinate where to compute the radius of curvature
    Output:
        radius of curvature at y_eval
    """
    
    # Fit new polynomials to x,y in world space
    fit_cr = np.polyfit(y*ym_per_pix, x*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    return ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])

def visualize_lanes(undistorted, y_values, left_fitx, right_fitx):
    """
    Draw a poligon that visualizes lines and drivable area
    Input:
        undistorted - undistorted image
        y_values - y values of the line curves
        left_fitx - x values of the left line curve
        right_fitx - x values of the right line curve
    Output: undistorted image with the visualization of lines and drivable area
    """
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(undistorted).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, y_values]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, y_values])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undistorted.shape[1], undistorted.shape[0])) 
    
    # Combine the result with the original image
    return cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)

def adjust_line(line, nonzerox, nonzeroy):
    """
    Adjust previously detected line to the new video frame
    Input:
        line - line detected in the previous video frame
        nonzerox - array of x coordinates of non-zero points in the video frame
        nonzeroy - array of y coordinates of non-zero points in the video frame
    Output: Line object if the previously detected line was successfully adjusted to the new video frame, None otherwise
    """
    
    margin = 50
    fit_value = line.current_fit[0]*(nonzeroy**2) + line.current_fit[1]*nonzeroy + line.current_fit[2]
    lane_inds = ((nonzerox > fit_value - margin) & (nonzerox < fit_value + margin))
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds]
    
    # Fit a second order polynomial 
    fit = np.polyfit(y, x, 2)
    
    # check if both lines are convex or both are concave
    # if not, fit the new line from scratch
    if fit[0] * line.current_fit[0] > 0:
        return Line(fit,x,y)
    else:
        return None

### Main function of the pipeline for finding lines and visualizing them

In [13]:
def find_line(img, undistorted, prev_lines=[], visualize=False):
    """
    Find lines and visualize them 
    Input:
        img - grayscale image
        undistorted - color image after camera calibration
        prev_line - list of previously found lines. This list can be emptry of there were are no previously found lines
        visualize - indicator if the function needs to visualize intermediate results
    Output:
        - undistorted image with the visualization of the lines and the drivable area
        - left line object
        - right line object
        - image with the visualization of intermediate results
    """
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
        
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((img, img, img))*255
    
    if len(prev_lines) > 0:
        left_line = adjust_line(prev_lines[0], nonzerox, nonzeroy)
        right_line = adjust_line(prev_lines[1], nonzerox, nonzeroy)
        
    if len(prev_lines) == 0 or left_line is None or right_line is None:        
        leftx_base, rightx_base = find_lane_base(img)
        left_line, out_img = track_lane(img, leftx_base, nonzerox, nonzeroy, out_img, visualize)
        right_line, out_img = track_lane(img, rightx_base, nonzerox, nonzeroy, out_img, visualize)
        
    # smooth the lines
    if len(prev_lines) > 0:
        left_line.current_fit = alpha * left_line.current_fit + (1-alpha) * prev_lines[0].current_fit
        right_line.current_fit = alpha * right_line.current_fit + (1-alpha) * prev_lines[1].current_fit
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0])
    left_fitx = left_line.compute_x(ploty)
    right_fitx = right_line.compute_x(ploty)
        
    if visualize == True:
        # Visualize lanes in image space
        out_img[left_line.y, left_line.x] = [255, 0, 0]
        out_img[right_line.y, right_line.x] = [0, 0, 255]
    
        in_image = (left_fitx >= 0) & (left_fitx < img.shape[1])
        out_img[ploty[in_image].astype(int), left_fitx[in_image].astype(int)] = [255,255,0]
        in_image = (right_fitx >= 0) & (right_fitx < img.shape[1])
        out_img[ploty[in_image].astype(int), right_fitx[in_image].astype(int)] = [255,255,0]
    
    result = visualize_lanes(undistorted, ploty, left_fitx, right_fitx)
    
    # Compute radius of curvature
    y_eval = np.max(ploty)
    left_curverad = radius_of_curvature(left_line.x, left_line.y, y_eval)
    right_curverad = radius_of_curvature(right_line.x, right_line.y, y_eval)  
    curvature = int(round((left_curverad+right_curverad)/2))
    string_radius_curvature = 'Radius of Curvature {}(m)'.format(curvature)
    
    # compute distance to center
    distance_to_center = (img.shape[1]/2 - (left_line.compute_x(y_eval) + right_line.compute_x(y_eval))/2)  * xm_per_pix
    if distance_to_center > 0:
        position = 'right'
    else: 
        position = 'left'
    string_position = 'Vehicle is {:.2f}m {} of the center'.format(abs(distance_to_center),position)
    
    # visualize radius of curvature and distance to center
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, string_radius_curvature,(10,70), font, 2, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(result, string_position,(10,140), font, 2, (255,255,255), 2, cv2.LINE_AA)
    
    if visualize == True:
        return result, left_line, right_line, out_img
    else:
        return result, left_line, right_line

### Find lines and drivable area in the test images and visualize them

In [14]:
# create directory of grayscale images with the visualization of lines
if not os.path.exists('output_images/fit_lane'):
    os.makedirs('output_images/fit_lane')

# create directory of undistorted images with the visualization of lines and drivable area
if not os.path.exists('output_images/final_image'):
    os.makedirs('output_images/final_image')

# Make a list of binary images
images = glob.glob('output_images/perspective_transform/*.jpg')

for fname in images:
    # read grayscale image after perspective transform
    img_color = cv2.imread(fname)
    img = cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY)
    
    # read undistorted color image
    undistorted = cv2.imread('output_images/undistorted_images/'+fname.split('\\')[-1])
    undistorted = cv2.cvtColor(undistorted,cv2.COLOR_BGR2RGB)
    
    # find lines, visualize lines and drivable area, create an image that visualizes intermediate results
    result, left_line, right_line, out_img = find_line(img, undistorted, visualize=True)
    
    # save images
    mpimg.imsave('output_images/fit_lane/'+fname.split('\\')[-1], out_img)
    mpimg.imsave('output_images/final_image/'+fname.split('\\')[-1], result)

## Test on Video 1

In [15]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### Pipeline for finding lines and visualizing them

In [16]:
# list of the lines found in previous frame
lines = []

def process_image_continuous(image):
    """
    Find lines in a video frame, visualize lines and drivable area
    Input: video frame
    Output: undistorted video frame with the visualization of lines and drivable area
    """
    
    global i, lines
    
    # undistort image
    undistorted = cv2.undistort(image, mtx, dist, None, mtx)
    
    # create thresholded image
    thresholded = create_binary_image(undistorted, s_thresh=(100, 255), sx_thresh=(20,100))
    
    # apply perspective transform
    warped = cv2.warpPerspective(thresholded, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    # find and visualize lane
    final, left_line, right_line = find_line(warped, undistorted, prev_lines = lines, visualize=False)
    
    # store lines for the reference in the next video frame
    lines = [left_line, right_line]
    
    return final

### Find lines in a video stream, visualize lines and drivable area

In [17]:
if not os.path.exists('output_videos'):
    os.makedirs('output_videos')
      
video_name = 'project_video.mp4'
clip = VideoFileClip(video_name)
clip_lanes = clip.fl_image(process_image_continuous)
clip_lanes.write_videofile('output_videos/' + video_name, audio=False)

[MoviePy] >>>> Building video output_videos/project_video.mp4
[MoviePy] Writing video output_videos/project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [03:09<00:00,  6.08it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video.mp4 



### Replay the video

In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('output_videos/' + video_name))